In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import openpyxl
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
import time

In [2]:
def get_player_position(driver):
    counter = 1
    while(True):
        if (driver.find_element(By.XPATH, "//*[@id='dvPlayerInfo']/div[{0}]".format(counter)).text == "Pozisyon"):
            return driver.find_element(By.XPATH, "//*[@id='dvPlayerInfo']/div[{0}]".format(counter + 1)).text.split(" ")[1]
        else:
            counter = counter + 2


In [3]:
def get_player_birth_day(driver):
    counter = 1
    while(True):
        if (driver.find_element(By.XPATH, "//*[@id='dvPlayerInfo']/div[{0}]".format(counter)).text == "D.Tarihi"):
            return driver.find_element(By.XPATH, "//*[@id='dvPlayerInfo']/div[{0}]".format(counter + 1)).text.split(" ")[2].replace("(", "").replace(")", "")
        else:
            counter = counter + 2

In [4]:
def get_player_weight(driver):
    counter = 1
    while(True):
        if (driver.find_element(By.XPATH, "//*[@id='dvPlayerInfo']/div[{0}]".format(counter)).text == "Kilo"):
            return driver.find_element(By.XPATH, "//*[@id='dvPlayerInfo']/div[{0}]".format(counter + 1)).text.split(" ")[1]
        else:
            counter = counter + 2

In [5]:
def get_player_height(driver):
    counter = 1
    while(True):
        if (driver.find_element(By.XPATH, "//*[@id='dvPlayerInfo']/div[{0}]".format(counter)).text == "Boy"):
            return driver.find_element(By.XPATH, "//*[@id='dvPlayerInfo']/div[{0}]".format(counter + 1)).text.split(" ")[1]
        else:
            counter = counter + 2

In [6]:
def get_player_league(driver):
    card_titles = driver.find_element(By.XPATH,"//*[@id='optaCard']/table/tbody/tr[1]").text.split(" ")
    card_word_size = len(card_titles)
    player_league = ""
    for i in range(card_word_size - 2):
        player_league += card_titles[i] + " "
    
    player_league.rstrip()
    
    return player_league
    

In [7]:
def get_player_season(driver):
    card_titles = driver.find_element(By.XPATH,"//*[@id='optaCard']/table/tbody/tr[1]").text.split(" ")
    player_season = card_titles[-2]

    return player_season

In [8]:
def get_player_stats(driver):
    try:
        player_stats_table = driver.find_element(By.XPATH,"//*[@id='optaCard']/table")
        player_stats_table_list = player_stats_table.text.split("\n") 
        player_stats = player_stats_table_list[1:]
        raw_player_stats = [item for i, item in enumerate(player_stats) if i % 2 == 1]
        
        return raw_player_stats
    except:
        return None

In [9]:
def get_club_carier(driver):
    try:
        player_club_carier = driver.find_element(By.XPATH,"//*[@id='playerFirstTab']/div/div[3]/div[2]/div[2]/table/tbody/tr[3]").text.split(" ")
        return player_club_carier
    except:
        return None

In [10]:
def get_country_club_carier(driver):
    try:
        player_country_club_carier = driver.find_element(By.XPATH,"//*[@id='playerFirstTab']/div/div[3]/div[2]/div[3]/table/tbody/tr[3]").text.split(" ")
        return player_country_club_carier
    except:
        return None

In [11]:
def get_workbook(file_name):
    try:
        workbook = openpyxl.load_workbook(file_name)
        return workbook
    except FileNotFoundError as err:
        print("File not found error: {0}".format(err))

In [12]:
def get_all_player_urls(workbook, row_starter = 1):
    player_urls = []
    worksheet = workbook.active
    for i in range(row_starter, worksheet.max_row + 1):
        player_urls.append(worksheet["A{0}".format(i)].value)
    return player_urls

In [13]:
def player_title(driver):
    player_info = {
        "player_name": "",
        "player_country": "",
        "player_team": "",
        "player_value": ""
    }

    player_xpath = {
        "player_name": "//*[@id='dvPlayerDetails']/div[2]/div[1]/div[1]/h1",
        "player_country": "//*[@id='dvPlayerDetails']/div[2]/div[1]/div[1]/div",
        "player_team": "//*[@id='dvPlayerDetails']/div[2]/div[1]/div[2]/div[1]/a/span",
        "player_value": "//*[@id='dvPlayerDetails']/div[2]/div[1]/div[2]/div[2]"
    }

    for key, xpath in player_xpath.items():
        try:
            player_info[key] = driver.find_element(By.XPATH, xpath).text
        except:
            player_info[key] = ""
    
    return player_info

In [14]:
def player_personal(driver):
    
    player_info = {
        "player_position": "",
        "player_birth_day": "",
        "player_weight": "",
        "player_height": ""
    }

    player_methods = {
        "player_position": "get_player_position",
        "player_birth_day": "get_player_birth_day",
        "player_weight": "get_player_weight",
        "player_height": "get_player_height"
    }

    for key, method in player_methods.items():
        try:
            player_info[key] = eval(method)(driver)
        except:
            player_info[key] = ""

    return player_info

In [15]:
def player_league_season(driver):
    
    player_info = {
        "player_league": "",
        "player_season": "",
    }

    player_methods = {
        "player_league": "get_player_league",
        "player_season": "get_player_season",
    }

    for key, method in player_methods.items():
        try:
            player_info[key] = eval(method)(driver)
        except:
            player_info[key] = ""

    return player_info

In [16]:
def player_stats(driver, player_position):

    player_info = {
        "player_match": "",
        "player_goal": "",
        "player_assist": "",
        "player_attemtp_per_match": "",
        "player_shots_on_goal_percentage_per_match": "",
        "player_pass_completion": "",
        "player_drib_percentage_per_match": "",
        "player_ball_winning_per_match": "",
        "player_winning_aerial_duels_per_match": "",
        "player_duel_win_percentage": "",
        "player_success_pass": "",
        "player_successful_crosses_per_match": ""
    }

    player_info_goalkeeper = {
        "player_match": "",
        "number_of_match_without_conceding_goal": "",
        "succes_rate_in_ball_ratention": "",
        "number_of_goal_conceded": "",
        "goal_conceded_from_inside_penalty_area": "",
        "number_save_from_inside_penalty_area": "",
        "goal_conceded_from_outside_penalty_area": "",
        "number_save_from_outside_penalty_area": "",
        "save_percentage": "",
        "save_percentage_in_penalty_area": "",
        "save_percentage_outside_penalty_area": "",
    }


    player_scrapped_state = {
        "player_match": "player_stats[0].split(' ')[0]",
        "player_goal": "player_stats[0].split(' ')[1]",
        "player_assist": "player_stats[0].split(' ')[2]",
        "player_attemtp_per_match": "player_stats[0].split(' ')[3].replace(',', '.')",
        "player_shots_on_goal_percentage_per_match": "player_stats[1].split(' ')[0].replace(',', '.')",
        "player_pass_completion": "player_stats[1].split(' ')[1].replace('%', '')",
        "player_drib_percentage_per_match": "player_stats[1].split(' ')[2].replace(',', '.')",
        "player_ball_winning_per_match": "player_stats[1].split(' ')[3].replace(',', '.')",
        "player_winning_aerial_duels_per_match": "player_stats[2].split(' ')[0].replace(',', '.')",
        "player_duel_win_percentage": "player_stats[2].split(' ')[1].replace('%', '')",
        "player_success_pass": "player_stats[2].split(' ')[2]",
        "player_successful_crosses_per_match": "player_stats[2].split(' ')[3].replace(',', '.')"
    }

    player_scrapped_state_goalkeeper = {
        "player_match": "player_stats[0].split(' ')[0]",
        "number_of_match_without_conceding_goal": "player_stats[0].split(' ')[1]",
        "succes_rate_in_ball_ratention": "player_stats[0].split(' ')[2].replace('%', '')",
        "number_of_goal_conceded": "player_stats[1].split(' ')[0]",
        "goal_conceded_from_inside_penalty_area": "player_stats[1].split(' ')[1]",
        "number_save_from_inside_penalty_area": "player_stats[1].split(' ')[3]",
        "goal_conceded_from_outside_penalty_area": "player_stats[1].split(' ')[4]",
        "number_save_from_outside_penalty_area": "player_stats[1].split(' ')[6]",
        "save_percentage": "player_stats[2].split(' ')[0].replace('%', '')",
        "save_percentage_in_penalty_area": "player_stats[2].split(' ')[1].replace('%', '')",
        "save_percentage_outside_penalty_area": "player_stats[2].split(' ')[2].replace('%', '')",
    }

    player_stats = get_player_stats(driver)

    if (player_position == "Kaleci"):
        player_info = player_info_goalkeeper
        player_scrapped_state = player_scrapped_state_goalkeeper

    for key, value in player_scrapped_state.items():
        try:
            player_info[key] = float(eval(value))
        except:
            player_info[key] = ""

    return player_info

In [17]:
def player_club_carier(driver):

    player_info = {
        "player_club_total_match": "",
        "player_club_total_goal": "",
        "player_club_total_yellow_card": "",
        "player_club_total_red_card": "",
    }

    player_club_carier = get_club_carier(driver)
    index = 0

    for key, value in player_info.items():
        try:
            player_info[key] = float(player_club_carier[index])
        except:
            player_info[key] = ""

        index = index + 1

    return player_info

In [18]:
def player_country_club_carier(driver):
    
    player_info = {
        "player_country_total_match": "",
        "player_country_total_goal": "",
        "player_country_total_yellow_card": "",
        "player_country_total_red_card": "",
    }

    player_country_club_carier = get_country_club_carier(driver)
    index = 0

    for key, value in player_info.items():
        try:
            player_info[key] = float(player_country_club_carier[index])
        except:
            player_info[key] = ""

        index = index + 1

    return player_info

In [ ]:
start_time = time.time()
file_name = 'soccer_players.xlsx'

workbook = get_workbook(file_name)
sheet = workbook.active

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

row_starter = 5011

player_urls = get_all_player_urls(workbook, row_starter)
print("--- %s get all player url seconds ---" % (time.time() - start_time))
start_time = time.time()

for i in range(0, len(player_urls)):
    try:
        url =  player_urls[i].strip()
        driver.get(url)
        print("--- %s get driver seconds ---" % (time.time() - start_time))
        start_time = time.time()
        player_title_info = player_title(driver)
        print("--- %s player_title_info seconds ---" % (time.time() - start_time))
        start_time = time.time()
        player_personal_info = player_personal(driver)
        print("--- %s player_personal_info seconds ---" % (time.time() - start_time))
        start_time = time.time()
        player_stats_info = player_stats(driver, player_personal_info["player_position"])
        print("--- %s player_stats_info seconds ---" % (time.time() - start_time))
        start_time = time.time()
        player_league_season_info = player_league_season(driver)
        print("--- %s player_league_season_info seconds ---" % (time.time() - start_time))
        start_time = time.time()
        player_club_carier_info = player_club_carier(driver)
        print("--- %s player_club_carier_info seconds ---" % (time.time() - start_time))
        start_time = time.time()
        player_country_club_carier_info = player_country_club_carier(driver)
        print("--- %s player_country_club_carier_info seconds ---" % (time.time() - start_time))
        start_time = time.time()
        player_info = {**player_title_info, **player_league_season_info, **player_personal_info, **player_stats_info, **player_club_carier_info, **player_country_club_carier_info}
        print("--- %s player_info seconds ---" % (time.time() - start_time))

        start_time = time.time()
        player_detail = list(player_info.values())
        print("--- %s player_detail seconds ---" % (time.time() - start_time))
        
        start_time = time.time()
        for i in range(2, len(player_detail) + 2):
            sheet.cell(row=row_starter, column=i, value = player_detail[i-2])

        workbook.save(file_name)
        print("--- %s player_detail_loop seconds ---" % (time.time() - start_time))
    
    except KeyboardInterrupt:
        workbook.save(file_name)
        driver.quit()
        print("Klavye")
        break
        
        
    except Exception as hata:
        print(f'Hata: {hata}')
        i -= 1
        print("Yusuf Haklıydı")
        continue

    if(not player_urls[i].strip()):
        break
        
    print(row_starter)
    row_starter += 1
    
    print("-----------------------------------------------------------------------\n")
    
driver.quit()

--- 8.679853916168213 get all player url seconds ---
--- 2.47910737991333 get driver seconds ---
--- 0.11868023872375488 player_title_info seconds ---
--- 0.6233334541320801 player_personal_info seconds ---
--- 0.02593088150024414 player_stats_info seconds ---
--- 0.029920101165771484 player_league_season_info seconds ---
--- 0.03690028190612793 player_club_carier_info seconds ---
--- 0.02094435691833496 player_country_club_carier_info seconds ---
--- 0.0 player_info seconds ---
--- 0.0 player_detail seconds ---
--- 1.5598299503326416 player_detail_loop seconds ---
5011
-----------------------------------------------------------------------

--- 2.5880820751190186 get driver seconds ---
--- 0.058843135833740234 player_title_info seconds ---
--- 0.708106279373169 player_personal_info seconds ---
--- 0.008975744247436523 player_stats_info seconds ---
--- 0.015958070755004883 player_league_season_info seconds ---
--- 0.01296544075012207 player_club_carier_info seconds ---
--- 0.0079786777

--- 1.5997159481048584 player_detail_loop seconds ---
5024
-----------------------------------------------------------------------

--- 3.1057791709899902 get driver seconds ---
--- 0.13364005088806152 player_title_info seconds ---
--- 0.46492815017700195 player_personal_info seconds ---
--- 0.0069811344146728516 player_stats_info seconds ---
--- 0.014960289001464844 player_league_season_info seconds ---
--- 0.014960289001464844 player_club_carier_info seconds ---
--- 0.0069806575775146484 player_country_club_carier_info seconds ---
--- 0.0 player_info seconds ---
--- 0.0 player_detail seconds ---
--- 1.445176362991333 player_detail_loop seconds ---
5025
-----------------------------------------------------------------------

--- 3.407404661178589 get driver seconds ---
--- 0.05285930633544922 player_title_info seconds ---
--- 0.5395584106445312 player_personal_info seconds ---
--- 0.022936582565307617 player_stats_info seconds ---
--- 0.03191661834716797 player_league_season_info seco

--- 1.5169460773468018 player_detail_loop seconds ---
5038
-----------------------------------------------------------------------

--- 5.467270135879517 get driver seconds ---
--- 0.2672884464263916 player_title_info seconds ---
--- 0.3999300003051758 player_personal_info seconds ---
--- 0.009973526000976562 player_stats_info seconds ---
--- 0.018949508666992188 player_league_season_info seconds ---
--- 0.01795029640197754 player_club_carier_info seconds ---
--- 0.007978439331054688 player_country_club_carier_info seconds ---
--- 0.0 player_info seconds ---
--- 0.0 player_detail seconds ---
--- 1.4960002899169922 player_detail_loop seconds ---
5039
-----------------------------------------------------------------------

--- 3.5440595149993896 get driver seconds ---
--- 0.07380056381225586 player_title_info seconds ---
--- 0.4602539539337158 player_personal_info seconds ---
--- 0.00797891616821289 player_stats_info seconds ---
--- 0.016953468322753906 player_league_season_info seconds 

--- 1.3902826309204102 player_detail_loop seconds ---
5052
-----------------------------------------------------------------------

--- 3.122551441192627 get driver seconds ---
--- 0.04986572265625 player_title_info seconds ---
--- 0.488694429397583 player_personal_info seconds ---
--- 0.017952680587768555 player_stats_info seconds ---
--- 0.025929689407348633 player_league_season_info seconds ---
--- 0.014960527420043945 player_club_carier_info seconds ---
--- 0.012965202331542969 player_country_club_carier_info seconds ---
--- 0.0 player_info seconds ---
--- 0.0 player_detail seconds ---
--- 1.3912804126739502 player_detail_loop seconds ---
5053
-----------------------------------------------------------------------

--- 3.0240318775177 get driver seconds ---
--- 0.06980514526367188 player_title_info seconds ---
--- 0.8806447982788086 player_personal_info seconds ---
--- 0.027924299240112305 player_stats_info seconds ---
--- 0.031056880950927734 player_league_season_info seconds ---


--- 1.4650819301605225 player_detail_loop seconds ---
5066
-----------------------------------------------------------------------

--- 3.9200820922851562 get driver seconds ---
--- 0.1486039161682129 player_title_info seconds ---
--- 0.27933454513549805 player_personal_info seconds ---
--- 0.005972623825073242 player_stats_info seconds ---
--- 0.01194143295288086 player_league_season_info seconds ---
--- 0.01334524154663086 player_club_carier_info seconds ---
--- 0.0069844722747802734 player_country_club_carier_info seconds ---
--- 0.0 player_info seconds ---
--- 0.0 player_detail seconds ---
--- 1.3992934226989746 player_detail_loop seconds ---
5067
-----------------------------------------------------------------------

--- 3.1913387775421143 get driver seconds ---
--- 0.12366938591003418 player_title_info seconds ---
--- 0.5704741477966309 player_personal_info seconds ---
--- 0.006997108459472656 player_stats_info seconds ---
--- 0.012950658798217773 player_league_season_info secon